## Simple LSTM Unit

+ Creating a Simple LSTM unit

+ Import necessary libraries, which are numpy and tensorflow

+ Import directly the tensorflow.contrib.rnn model which includes the functions for building RNNs.

In [1]:
import numpy as np
import tensorflow as tf

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/conda/envs/python/lib/python3

Initiate a Session

In [2]:
sess = tf.Session()

Create a Network that only have one LSTM cell

+ pass two elements
    + prv_output (h)
    + prv_state (c)
    


+ Initialize a state vector, Here state is a tuple with two elements each one is of size [1x4]

    + one for passing prv_output to the next time step
    + another for passing prv_state to next time stamp
    

In [6]:
# Outpu size (dimension), which is same as hidden size in the cell
LSTM_CELL_SIZE = 4

lstm_cell = tf.contrib.rnn.BasicLSTMCell(LSTM_CELL_SIZE, state_is_tuple = True)
print(lstm_cell)

state = (tf.zeros([1, LSTM_CELL_SIZE]),) * 2
print(state)

(<tf.Tensor 'zeros:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'zeros:0' shape=(1, 4) dtype=float32>)


Define a simple input,
+ batch_size = 1
+ seq_len = 6

In [7]:
simple_input = tf.constant([[3,2,2,2,2,2]], dtype = tf.float32)
print(sess.run(simple_input))

[[3. 2. 2. 2. 2. 2.]]


Now pass the input to the LSTM Cell. and check the new state

In [11]:
with tf.variable_scope("LSTM-sample1"):
    output, state_new = lstm_cell(simple_input, state)
    
sess.run(tf.global_variables_initializer())
print(sess.run(state_new))

LSTMStateTuple(c=array([[ 0.12857103,  0.5890479 , -0.81194913,  0.06806316]],
      dtype=float32), h=array([[ 0.00972666,  0.19999017, -0.05892208,  0.00235921]],
      dtype=float32))


There will be two parts,

+ new state c,
+ the output h

Check the output Again

In [12]:
sess.run(output)

array([[ 0.00972666,  0.19999017, -0.05892208,  0.00235921]],
      dtype=float32)

And its an array

## Stacked LSTM

Create a 2-Layer LSTM

Start with a new Session

In [13]:
sess = tf.Session()

In [14]:
input_dim = 6

Create the Stacked Cell

In [15]:
cells = []

Create the first layer of the LSTM cell

In [16]:
LSTM_CELL_SIZE_1 = 4 # Four hidden nodes

cell1 = tf.contrib.rnn.LSTMCell(LSTM_CELL_SIZE_1)
cells.append(cell1)

Creating the Second Layer

In [17]:
LSTM_CELL_SIZE_2 = 7 # 7 hidden nodes

cell2 = tf.contrib.rnn.LSTMCell(LSTM_CELL_SIZE_2)
cells.append(cell2)

Now create stacked LSTM

+ use tf.contrib.rnn.MultiRNNCell function

+ It takes multiple single layer LSTM cells to create stacked ones.

In [18]:
stacked_lstm = tf.contrib.rnn.MultiRNNCell(cells)

In [20]:
stacked_lstm

Create an RNN from stacked_lstm

In [21]:
# batch_size x time steps x features

data = tf.placeholder(tf.float32, [None, None, input_dim])

output, state = tf.nn.dynamic_rnn(stacked_lstm, data, dtype = tf.float32)

Say, 
+ input sequence length = 3
+ Dimensionality of input = 6
+ input should be a tensor shape [batch_size, max_time, dimension]
+ (2,3,6)

In [23]:
# Batch size x time steps x features

sample_input = [[[1,2,3,4,5,6],[2,3,4,5,6,7],[3,4,5,6,7,8]],[[6,5,4,3,2,1],[7,6,5,4,3,2],[8,7,6,5,4,3]]]

print(sample_input)

[[[1, 2, 3, 4, 5, 6], [2, 3, 4, 5, 6, 7], [3, 4, 5, 6, 7, 8]], [[6, 5, 4, 3, 2, 1], [7, 6, 5, 4, 3, 2], [8, 7, 6, 5, 4, 3]]]


Send the input to the network and Check the Output

In [24]:
output

<tf.Tensor 'rnn/transpose_1:0' shape=(?, ?, 7) dtype=float32>

In [25]:
state

(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 4) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 4) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 7) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_6:0' shape=(?, 7) dtype=float32>))

In [26]:
sess.run(tf.global_variables_initializer())

In [27]:
sess.run(output, feed_dict = {data : sample_input})

array([[[ 6.2447362e-02, -4.2567248e-03,  4.1373175e-02, -2.7059823e-02,
         -2.6844686e-02,  5.6483757e-02, -2.1938784e-02],
        [ 1.2336028e-01, -1.0441951e-02,  8.0343835e-02, -5.4756321e-02,
         -5.9932973e-02,  1.0962753e-01, -4.2171191e-02],
        [ 1.6638190e-01, -1.4044318e-02,  1.0887790e-01, -7.9413399e-02,
         -9.0599865e-02,  1.4862216e-01, -5.5209666e-02]],

       [[ 3.7793638e-03,  5.9216499e-04,  2.7222617e-03, -2.8597047e-03,
         -2.0355368e-03,  3.8535472e-03, -4.7673407e-04],
        [ 9.4983429e-03, -2.9874622e-05,  5.9002684e-03, -5.5827643e-03,
         -4.4475645e-03,  9.2490287e-03, -2.3975086e-03],
        [ 1.6734524e-02, -9.8800263e-04,  9.4701257e-03, -8.8657271e-03,
         -7.5431662e-03,  1.5503475e-02, -4.8493315e-03]]], dtype=float32)

The out